# R IRIS Model

 * Wrap a scikit-learn python model for use as a prediction microservice in seldon-core
   * Run locally on Docker to test
   * Deploy on seldon-core running on minikube
 
## Depenencies

 * [Helm](https://github.com/kubernetes/helm)
 * [Minikube](https://github.com/kubernetes/minikube)
 * [S2I](https://github.com/openshift/source-to-image)
 * R

```bash
pip install sklearn
pip install grpcio-tools
```

## Train locally
 

In [ ]:
!Rscript train.R

Wrap model using s2i

In [ ]:
!s2i build . seldonio/seldon-core-s2i-r:0.1 r-iris:0.1

In [ ]:
!docker run --name "iris_predictor" -d --rm -p 5000:5000 r-iris:0.1

In [ ]:
!cd ../../../wrappers/testing && make build_protos

Send some random features that conform to the contract

In [ ]:
!python ../../../wrappers/testing/tester.py contract.json 0.0.0.0 5000 -p

In [ ]:
!docker rm iris_predictor --force

# Test using Minikube

**Due to a [minikube/s2i issue](https://github.com/SeldonIO/seldon-core/issues/253) you will need Minikube version 0.25.2**

In [ ]:
!minikube start --memory 4096 --feature-gates=CustomResourceValidation=true --extra-config=apiserver.Authorization.Mode=RBAC

In [ ]:
!kubectl create clusterrolebinding kube-system-cluster-admin --clusterrole=cluster-admin --serviceaccount=kube-system:default

In [ ]:
!helm init

In [ ]:
!kubectl rollout status deploy/tiller-deploy -n kube-system

In [ ]:
!helm install ../../../helm-charts/seldon-core-crd --name seldon-core-crd  --set usage_metrics.enabled=true
!helm install ../../../helm-charts/seldon-core --name seldon-core 

In [ ]:
!eval $(minikube docker-env) && s2i build . seldonio/seldon-core-s2i-r:0.1 r-iris:0.1

In [ ]:
!kubectl create -f r_iris_deployment.json

Wait until ready (replicas == replicasAvailable)

In [ ]:
!kubectl get seldondeployments seldon-deployment-example -o jsonpath='{.status}' 

In [ ]:
!cd ../../../util/api_tester && make build_protos 

In [ ]:
!python ../../../util/api_tester/api-tester.py contract.json \
    `minikube ip` `kubectl get svc -l app=seldon-apiserver-container-app -o jsonpath='{.items[0].spec.ports[0].nodePort}'` \
    --oauth-key oauth-key --oauth-secret oauth-secret -p

In [ ]:
!minikube delete